In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
df = pd.read_csv("gdrive/My Drive/twitter/twitter_data.csv", encoding="ISO-8859-1")

In [4]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [5]:
df.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [6]:
df["text"] = df["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]
df["sentiment"] = df["0"]
df["user"] = df["_TheSpecialOne_"]
df["user_id"] = df["1467810369"]

In [7]:
del df["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]
del df["_TheSpecialOne_"]
del df["1467810369"]
del df["NO_QUERY"]
del df["0"]

In [8]:
df.head()

,Mon Apr 06 22:19:45 PDT 2009,text,sentiment,user,user_id
0,Mon Apr 06 22:19:49 PDT 2009,is upset that he can't update his Facebook by ...,0,scotthamilton,1467810672
1,Mon Apr 06 22:19:53 PDT 2009,@Kenichan I dived many times for the ball. Man...,0,mattycus,1467810917
2,Mon Apr 06 22:19:57 PDT 2009,my whole body feels itchy and like its on fire,0,ElleCTF,1467811184
3,Mon Apr 06 22:19:57 PDT 2009,"@nationwideclass no, it's not behaving at all....",0,Karoli,1467811193
4,Mon Apr 06 22:20:00 PDT 2009,@Kwesidei not the whole crew,0,joy_wolf,1467811372


In [9]:
df["sentiment"].value_counts()

4    800000
0    799999
Name: sentiment, dtype: int64

In [10]:
# 4 is positive
# 0 is negative

In [11]:
from sklearn.model_selection import train_test_split

X = df[['text']]
y = df['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [12]:
from bs4 import BeautifulSoup   
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text()
    # 2. Convert to lower case
    lower_case = review_text.lower()
    # 3. Tokenize into words, removing punctuation
    retokenizer = RegexpTokenizer(r'[a-z]+')
    words = retokenizer.tokenize(lower_case)
    # 4a. In Python, searching a set is much faster than searching a list, so convert the stop words to a set
    stops = set(stopwords.words('english'))
    # 4b. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    # 6. Join the words back into one string separated by space, and return the result.
    return(" ".join(meaningful_words))

In [14]:
X_train['text'] = X_train['text'].apply(lambda x: review_to_words(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
X_test['text'] = X_test['text'].apply(lambda x: review_to_words(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object 
vectorizer = CountVectorizer() 

# Fits the model and learn the vocabulary
train_data_features = vectorizer.fit(X_train['text'])

# Transform our training data into feature vectors
train_data_features = vectorizer.transform(X_train['text'])

# Transform our test data based on the learned vocabulary from the training data
test_data_features = vectorizer.transform(X_test['text'])

In [17]:
print(train_data_features.shape)

(1199999, 487838)


In [18]:
print(test_data_features.shape)

(400000, 487838)


In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_data_features, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
model.score(test_data_features, y_test)

0.7778425

In [21]:
import numpy as np

new_review = '<p>This is a $100 negative review</p>. I didnt like it. It was bad. The actors were terrible. That movie was fucking bullshit'

cleaned_review = review_to_words(new_review)

df_review = pd.DataFrame([cleaned_review])

review_transformed = vectorizer.transform(df_review[0])

model.predict(review_transformed)[0]

0

In [22]:
new_review = '<p>This is a $100 positive review</p>. I didnt love it. It was great. The actors were amazing. That movie was fucking awesome'

cleaned_review = review_to_words(new_review)

df_review = pd.DataFrame([cleaned_review])

review_transformed = vectorizer.transform(df_review[0])

model.predict(review_transformed)[0]

4

In [23]:
pip install catboost

In [24]:
from catboost import CatBoostClassifier

In [25]:
model = CatBoostClassifier()
model.fit(train_data_features, y_train)

Learning rate set to 0.212689
0:	learn: 0.6788120	total: 2.74s	remaining: 45m 40s
1:	learn: 0.6695289	total: 5.26s	remaining: 43m 47s
2:	learn: 0.6626022	total: 7.79s	remaining: 43m 8s
3:	learn: 0.6574156	total: 10.3s	remaining: 42m 44s
4:	learn: 0.6525795	total: 12.8s	remaining: 42m 22s
5:	learn: 0.6478080	total: 15.3s	remaining: 42m 16s
6:	learn: 0.6434979	total: 17.9s	remaining: 42m 17s
7:	learn: 0.6400060	total: 20.4s	remaining: 42m 11s
8:	learn: 0.6372267	total: 22.9s	remaining: 42m 6s
9:	learn: 0.6343040	total: 25.6s	remaining: 42m 19s
10:	learn: 0.6319186	total: 28.1s	remaining: 42m 10s
11:	learn: 0.6294279	total: 30.6s	remaining: 42m 2s
12:	learn: 0.6274471	total: 33.1s	remaining: 41m 56s
13:	learn: 0.6252497	total: 35.6s	remaining: 41m 49s
14:	learn: 0.6229999	total: 38.4s	remaining: 42m 2s
15:	learn: 0.6212703	total: 40.9s	remaining: 41m 57s
16:	learn: 0.6194106	total: 43.8s	remaining: 42m 10s
17:	learn: 0.6175914	total: 46.5s	remaining: 42m 16s
18:	learn: 0.6159335	total: 49

In [26]:
model.score(test_data_features, y_test)

0.7694425